In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [2]:
path = './UJIIndoorLoc/trainingData2.csv'
train_df = pd.read_csv(path,header=0)
train_ap_features = scale(np.asarray(train_df.iloc[:,:520]).astype(float),axis = 1)
train_df['REFPOINT'] = train_df.apply(lambda row: str(int(row['SPACEID'])) + str(int(row['RELATIVEPOSITION'])),
                                  axis=1)

In [4]:
train_labels = np.asarray(pd.get_dummies(train_df['REFPOINT']))
OUTPUT_DIM = train_labels.shape[1]
OUTPUT_DIM

213

In [5]:
train_test_split = np.random.rand(len(train_ap_features)) < 0.9
x_train_val = train_ap_features[train_test_split]
y_train_val = train_labels[train_test_split]
x_test = train_ap_features[~train_test_split]
y_test = train_labels[~train_test_split]

train_val_split = np.random.rand(len(x_train_val)) < 0.9
x_train = x_train_val[train_val_split]
y_train = y_train_val[train_val_split]
x_val = x_train_val[~train_val_split]
y_val = y_train_val[~train_val_split]

# x_train.shape,y_train.shape
print(len(x_train), '训练数据')   #  训练数据 
print(len(x_val), '验证数据 ')    #  验证数据 
print(len(x_test), '测试数据')    #  测试数据

16118 训练数据
1818 验证数据 
2001 测试数据


In [21]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
model = Sequential()
sae_hidden_layers= [256,128,64,128,256]
model.add(Dense(sae_hidden_layers[0],input_dim=520,activation='relu',use_bias=False))
for units in sae_hidden_layers[1:]:
    model.add(Dense(units, activation='relu', use_bias=False))
model.add(Dense(OUTPUT_DIM, activation='sigmoid', use_bias=False))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=50, verbose=1)

Epoch 1/50
252/252 [==============================] - 1s 3ms/step - loss: 0.0517 - accuracy: 0.0373 - val_loss: 0.0239 - val_accuracy: 0.0715
Epoch 2/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0227 - accuracy: 0.1384 - val_loss: 0.0188 - val_accuracy: 0.2074
Epoch 3/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0188 - accuracy: 0.2677 - val_loss: 0.0160 - val_accuracy: 0.3003
Epoch 4/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0162 - accuracy: 0.3736 - val_loss: 0.0143 - val_accuracy: 0.3823
Epoch 5/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0145 - accuracy: 0.4622 - val_loss: 0.0130 - val_accuracy: 0.4450
Epoch 6/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0132 - accuracy: 0.5359 - val_loss: 0.0122 - val_accuracy: 0.4989
Epoch 7/50
252/252 [==============================] - 1s 2ms/step - loss: 0.0121 - accuracy: 0.5960 - val_loss: 0.0113 - val_accuracy: 0.5600
Epoch 

In [22]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 256)               133120    
                                                                 
 dense_27 (Dense)            (None, 128)               32768     
                                                                 
 dense_28 (Dense)            (None, 64)                8192      
                                                                 
 dense_29 (Dense)            (None, 128)               8192      
                                                                 
 dense_30 (Dense)            (None, 256)               32768     
                                                                 
 dense_31 (Dense)            (None, 213)               54528     
                                                                 
Total params: 269,568
Trainable params: 269,568
Non-tr

In [23]:
preds = model.predict(x_test, batch_size=32)
results = np.equal(np.argmax(y_test, axis=1), np.argmax(preds, axis=1)).astype(int)
acc = results.mean()
acc

63/63 [==============================] - 0s 756us/step


0.7096451774112944